# Sentiment Analysis 2 with textblob

In [1]:
import os
from GoogleNews import GoogleNews
from textblob import TextBlob
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from openbb import obb

In [2]:
def save_data(data):
    output_dir = "data"
    # os.makedirs(output_dir, exist_ok=True)

    # Get the date and actual hour
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    file_path = os.path.join(output_dir, f"data_{timestamp}.csv")
    
    # Save the DATA
    data.to_csv(file_path, index=True)
    print(f"Data saved in: {file_path}")

In [3]:
def fetch_feed(actif):
    #date in format "mm/dd/yyyy"
    time.sleep(30)
    googlenews=GoogleNews()
    googlenews.set_lang('en')
    googlenews.set_period('7d')
    #googlenews.set_time_range(start_date, end_date)
    googlenews.set_encode('utf-8')
    googlenews.search(actif)

    # process
    news_items = []
    pages = 1
    summary = []
    positive_count = 0
    negative_count = 0
    neutral_count = 0
    subjectivity_count = 0
    objectivity_count = 0
    
    for page in range(1, pages + 1):
        time.sleep(30)
        print(f"Raul Aviles the Goat Get news of {page} page :) ...")
        #googlenews.get_page(page)
        news=googlenews.results()
        
        for entry in news:
            title = entry['title']
            desc = entry['desc']
            information = f"{title}. {desc}"
            text=TextBlob(information)
            
            # obtain polarity
            sentiment = text.sentiment.polarity
            subjectivity = text.sentiment.subjectivity

            # Sentiment threshold
            if sentiment > 0.05:
                positive_count += 1
            elif sentiment <= -0.02:
                negative_count += 1
            else:
                neutral_count += 1

            # Subjectivity threshold
            if subjectivity >= 0.5:
                subjectivity_count += 1
            else:
                objectivity_count += 1
        
            news_items.append([entry['datetime'], information, sentiment, subjectivity])

    df = pd.DataFrame(news_items, columns=['Fecha', 'Noticia', 'Sentiment_Polarity', 'Sentiment_Subjectivity'])
    df = df.set_index('Fecha')    
    summary.append({
        'Positive_Ratio': positive_count/len(news_items), 
        'Negative_Ratio': negative_count/len(news_items), 
        'Neutral_Ratio': neutral_count/len(news_items),
        'Positive Count': positive_count,
        'Negative Count': negative_count,
        'Neutral Count': neutral_count,
        'Subjectivity_Ratio': subjectivity_count / (subjectivity_count + objectivity_count),
        'Objectivity_Ratio': objectivity_count / (subjectivity_count + objectivity_count)
    })
    
    save_data(df)
    
    return summary
    

In [4]:
x = fetch_feed("GOOGL")

Raul Aviles the Goat Get news of 1 page :) ...
Data saved in: data\data_20241211_100555.csv


In [5]:
x

[{'Positive_Ratio': 0.45454545454545453,
  'Negative_Ratio': 0.09090909090909091,
  'Neutral_Ratio': 0.45454545454545453,
  'Positive Count': 5,
  'Negative Count': 1,
  'Neutral Count': 5,
  'Subjectivity_Ratio': 0.36363636363636365,
  'Objectivity_Ratio': 0.6363636363636364}]

Subjetividad -> Menor a 0.5: Predominantemente objetivo (más basado en hechos).
Mayor o igual a 0.5: Predominantemente subjetivo (más basado en opiniones).